# Run An Odds Ratio

### Authors: Calvin Howard.

#### Last updated: July 6, 2023

Use this to run/test a statistical model on a spreadsheet.

Notes:
- To best use this notebook, you should be familar with GLM design and Contrast Matrix design. See this webpage to get started:
[FSL's GLM page](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM)

# 00 - Import CSV with All Data
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

Prep Output Direction

In [ ]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Research/2023/subiculum_cognition_and_age/figures/Figures/odds_ratios'

Import Data

In [ ]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Dropbox (Partners HealthCare)/studies/cognition_2023/metadata/master_list_proper_subjects.xlsx'
sheet = 'master_list_proper_subjects'

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()

# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [ ]:
drop_list = ['City', 'StimMatch', 'Cognitive_Improve']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
display(data_df)

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [ ]:
data_df.columns

Set the parameters for dropping rows

In [ ]:
column = 'City'  # The column you'd like to evaluate
condition = 'equal'  # The condition to check ('equal', 'above', 'below', 'not')
value = 'BWH' # The value to drop if found

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
display(data_df)

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [ ]:
# Remove anything you don't want to standardize
cols_not_to_standardize = None # ['Z_Scored_Percent_Cognitive_Improvement_By_Origin_Group', 'Z_Scored_Subiculum_T_By_Origin_Group_'] #['Age']

In [ ]:
data_df = cal_palm.standardize_columns(cols_not_to_standardize)
data_df

In [ ]:
# for col in data_df.columns:
#     if 'CSF' and 'eh' not in col:
#         data_df[col] = data_df[col] * -1

# 02 - Simple Odds Ratio


In [ ]:
import pandas as pd

def calculate_contingency_values(df, condition_column, outcome_column, condition_success, outcome_success):
    """
    Calculates the values of a, b, c, and d for a 2x2 contingency table from a DataFrame.

    Parameters:
    df (pd.DataFrame): The input data frame.
    condition_column (str): The name of the column that contains the condition.
    outcome_column (str): The name of the column that contains the outcome.
    condition_success (any): The value in the condition column that indicates success.
    outcome_success (any): The value in the outcome column that indicates success.

    Returns:
    tuple: A tuple containing the values of a, b, c, and d.
    """
    # Patients who meet the condition
    condition_positive = df[condition_column] == condition_success

    # Patients who don't meet the condition
    condition_negative = ~condition_positive

    # Patients who improved
    outcome_positive = df[outcome_column] == outcome_success

    # Patients who did not improve
    outcome_negative = ~outcome_positive

    # Count values for a, b, c, d
    a = df[condition_positive & outcome_positive].shape[0]
    b = df[condition_positive & outcome_negative].shape[0]
    c = df[condition_negative & outcome_positive].shape[0]
    d = df[condition_negative & outcome_negative].shape[0]

    return a, b, c, d

# Example usage:
# df = pd.DataFrame({'Condition': [...], 'Outcome': [...]})
# a, b, c, d = calculate_contingency_values(df, 'Condition', 'Outcome', 'ConditionSuccessValue', 'OutcomeSuccessValue')
# print(a, b, c, d)

def calculate_odds_ratio(a, b, c, d):
    """
    Calculates the odds ratio from a 2x2 contingency table.
    
    Parameters:
    a (int): Number of patients who meet the condition and improved.
    b (int): Number of patients who meet the condition and did not improve.
    c (int): Number of patients who do not meet the condition but improved.
    d (int): Number of patients who do not meet the condition and did not improve.
    
    Returns:
    float: The odds ratio.
    """
    try:
        odds_ratio = (a * d) / (b * c)
        return odds_ratio
    except ZeroDivisionError:
        return "Cannot calculate odds ratio due to division by zero."

# Example usage:
# a, b, c, d = 10, 20, 30, 40
# print(calculate_odds_ratio(a, b, c, d)


In [ ]:
data_df.columns

In [ ]:
a, b, c, d = calculate_contingency_values(data_df, condition_column='StimMatch', outcome_column='Cognitive_Outcome', condition_success='Match', outcome_success='improve')
calculate_odds_ratio(a, b, c, d)

# 03 - Forest Plot
- This code allows you to adjust for covariates.
- Instead of a normal ad/cb odds ratio calculation, this uses logistic regression.

In [ ]:
data_df.columns

In [ ]:
data_df['Cognitive_Improve']

In [ ]:
outcome = 'Cognitive_Improve'
predictor = 'StimMatch'
cohorts = 'City'
covariate_list = None

In [ ]:
from calvin_utils.statistical_utils.forest_plots import OddsRatioForestPlot
# Create an instance of the LogisticOddsRatioPlot class with covariates
plot = OddsRatioForestPlot(data_df, outcome_col=outcome, predictor_col=predictor,
                             category_col=cohorts, covariates=None, table=False, log_odds=True, out_dir=out_dir)

# Run the plotting process
plot.run()
plot.data_for_plot